In [32]:
from Plan import CreateSamples_renewed
from Prepare import OT2Commands as ALH
from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib

In [105]:
importlib.reload(CreateSamples_renewed)
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

In [59]:
path = r"Testing Plans\05_10_21_CdSe_Ligand_Study.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [60]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'],
                                                                     unity_filter=False)
concentration_df

,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity
0,0.0,0.0,0.00,0.00
1,0.0,0.0,0.00,0.25
2,0.0,0.0,0.00,0.50
3,0.0,0.0,0.25,0.00
4,0.0,0.0,0.25,0.25
...,...,...,...,...
76,0.1,0.1,0.25,0.25
77,0.1,0.1,0.25,0.50
78,0.1,0.1,0.50,0.00
79,0.1,0.1,0.50,0.25


In [117]:
complete_amounts_added = CreateSamples_renewed.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
stock_dict = CreateSamples_renewed.stock_dictionary(plan['Stock Names'], plan['Stock Concentration Units'], plan['Stock Concentrations'])
complete_df = CreateSamples_renewed.calculate_stock_volumes_from_component_concs(plan, complete_amounts_added, stock_dict)
complete_df= CreateSamples_renewed.complete_missing_volume_with_commmon_solvent(plan['Sample Amount'], complete_df, stock_dict, 'Octadecene')
complete_df_uL = CreateSamples_renewed.convert_mL_to_uL(complete_df)
stock_volumes = CreateSamples_renewed.isolate_common_column(complete_df_uL, 'stock')
stock_volumes = stock_volumes[0:3]
stock_volumes

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units


,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,Octadecene-stock volume uL
0,0.0,0.0,0.0,0.000000,5000.000000
1,0.0,0.0,0.0,411.269988,4588.730012
2,0.0,0.0,0.0,822.539975,4177.460025


In [118]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.3', extra_labware=custom_labware_dict) # make note this must be run constantly ~ if say making loaded dict twice
loaded_dict = ALH.loading_labware(protocol, plan)

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


In [119]:
max_source_vol = 18000 # if the list index reports out of range most likely not enough slots aka too much volume,
# make it so you can mix labwares max volumes should be automatic in checking, maybe use the end the labware name to grab the uL
# like it would see that the volume uses more than one entire well plate for stocks then would reccomend and use the larger well plate, wont use it if no necessary
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 
# len(stock_position_info['Octadecene-stock volume uL']['Ranges'])
stock_position_info

{'CdAc-TOP-stock amount volume uL': {'Ranges': [[0, 3]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Se-TOP-stock amount volume uL': {'Ranges': [[0, 3]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'OleicAc-stock amount volume uL': {'Ranges': [[0, 3]],
  'Stock Wells': [A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Oleylamine-stock amount volume uL': {'Ranges': [[0, 3]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Octadecene-stock volume uL': {'Ranges': [[0, 3]],
  'Stock Wells': [B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]}}

In [120]:
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict) 
# these directions should be used to make the df, unless there is a final move??

In [121]:
ALH.pipette_volumes_sample_wise(protocol, directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 0.0 from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Delaying for 0 minutes and 5.0 seconds
Returning tip
Dropping tip into A1 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A2 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 0.0 from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Delaying for 0 minutes and 5.0 seconds
Returning tip
Dropping tip into A2 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A3 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 0.0 from A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Delaying for 0 minutes and 5.0 seconds
Returning tip
Dropping tip into A3 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A4 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 0.0 from A4 of 20mLscintillation 12 Well Plate 